# Get Strava Client (Athlete)

In [1]:
!pip install stravalib --quiet
from stravalib import Client

from google.colab import userdata
import sys
import logging
import pandas as pd
import numpy as np

logging.getLogger("stravalib.util.limiter").setLevel(logging.ERROR)    # suppress stravalib warnings
pd.set_option("display.max_columns", None)  # view ALL columns
# pd.reset_option("display.max_columns")      # to reset to default (if needed)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 6.3 MB/s eta 0:00:00


  return datetime.utcnow().replace(tzinfo=utc)



In [2]:
token = userdata.get("GITHUB_STRAVA_REPO_TOKEN")
username = "anw-g01"
repo = "strava-data-analysis"

!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'strava-data-analysis'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 13 (delta 3), reused 12 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), done.
Resolving deltas: 100% (3/3), done.


In [3]:
sys.path.append(f"/content/{repo}")    # add directory containing oauth.py
from oauth import get_athlete   # utilise function from oauth.py module (see GitHub repo)

# refresh an access token, authenticate the athlete, and return an authorised client:
token_response = get_athlete(
    client_id=userdata.get("CLIENT_ID"),
    client_secret=userdata.get("CLIENT_SECRET"),
    refresh_token=userdata.get("REFRESH_TOKEN1"),
)


Successfully authenticated athlete: Anwarat Gurung

{
    "bound_client": "<stravalib.client.Client object at 0x7ac43b3a0800>",
    "id": 36605442,
    "city": "Cambridge",
    "country": "",
    "created_at": "2018-11-19 12:35:26+00:00",
    "firstname": "Anwarat",
    "lastname": "Gurung",
    "premium": false,
    "profile": "https://dgalywyr863hv.cloudfront.net/pictures/athletes/36605442/12211903/7/large.jpg",
    "profile_medium": "https://dgalywyr863hv.cloudfront.net/pictures/athletes/36605442/12211903/7/medium.jpg",
    "resource_state": 3,
    "sex": "M",
    "state": "England",
    "summit": false,
    "updated_at": "2025-12-17 23:39:58+00:00",
    "bikes": [],
    "clubs": [
        "bound_client=None id=231407 name='The Strava Club' resource_state=2 activity_types=[] city='San Francisco' country='United States' cover_photo='https://dgalywyr863hv.cloudfront.net/pictures/clubs/231407/5098428/21/large.jpg' cover_photo_small='https://dgalywyr863hv.cloudfront.net/pictures/clubs/